<a href="https://colab.research.google.com/github/PyGeek03/Crazy-Fibonacci/blob/master/Fibonacci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will need decimal because 1) Python represents very big numbers in the scientific notation and 2) if we use ordinary float, at n = 1474 (which is pitifully small), it will overflow:  `OverflowError: (34, 'Numerical result out of range')`

In [0]:
from timeit import default_timer
from math import sqrt

from decimal import *
D = Decimal

Let's initialize the range of decimal's possible exponents to be as wide as necessary:

In [0]:
# fib(0) = 0
getcontext().Emin = 0

# On 64-bit machines, decimal.MAX_EMAX is 999999999999999999. On 32-bit ones, ut's 425000000.
getcontext().Emax = MAX_EMAX

At n < 4784972, we use Binet's formula. But at n = 4784972, decimal will overflow, so we have to resort to fast doubling:

In [0]:
def fib(n):
    assert n >= 0
    if n < 4784972: 
        return _binet(n)
    else:
        return _fast_doubling(n)[0]
  
  
def _binet(n):
    """Binet's formula:
    (golden**n - (1-golden)**n)/sqrt(5)
    """   
    golden = (1 + D(5).sqrt())/2
    fib = (Context().power(golden,D(n)) - Context().power(1-golden, D(n)))/D(5).sqrt()
    return Context().to_integral_exact(fib)
  
  
def _fast_doubling(n):
    """Fast doubling method:
    F(2n) = F(n)*(2*F(n+1) – F(n))
    F(2n + 1) = F(n)**2 + F(n+1)**2
    """
    if n <= 4784970:
        return (_binet(n), _binet(n+1))
    else:
        fib_n, fib_n_plus_1 = _fast_doubling(n // 2)
        fib_2n = fib_n * (fib_n_plus_1 * 2 - fib_n)
        fib_2n_plus_1 = fib_n**2 + fib_n_plus_1**2
        fib_2n_plus_2 = fib_2n + fib_2n_plus_1
      
        if n % 2 == 0:
            return (fib_2n, fib_2n_plus_1)
        else:
            return (fib_2n_plus_1, fib_2n_plus_2)

Let's look at the maximum Fibonacci number our fib function could calculate, and how long the calculation would take!

In [49]:
max_n = 4784971966781665971  # Any bigger n and decimal will overflow

start = default_timer()
max_fib = fib(max_n)
taken = default_timer() - start
print('The {}-th Fibonacci number is:\n{}\n'.format(max_n, max_fib))
print('Calculating it takes {} seconds'.format(taken))

The 4784971966781665971-th Fibonacci number is:
3.764070649112636815213426347E+999999999999999999

Calculating it takes 0.00029197200001362944 seconds


That's less than a fraction of a second! However, it's not absolutely accurate though. The last n that would give us absolute accuracy is 122. 

Try that out for yourself!

(Source for expected Fibonacci numbers: http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibtable.html)

In [0]:
actual   = fib(121)
expected =  8670007398507948658051921

In [0]:
actual   = fib(122)  # The last fib(n) with absolute accuracy
expected = 14028366653498915298923761

In [0]:
actual   = fib(123)  # It's all downhill from here...
expected = 22698374052006863956975682

In [0]:
actual   = fib(124)
expected = 36726740705505779255899443

In [69]:
print('Actual: {}'.format(actual))
print('Expected: {}'.format(expected))
print('Difference: {}'.format(actual - expected))

Actual: 36726740705505779255899442
Expected: 36726740705505779255899443
Difference: -1
